# Automating the georeferencing process
The notebook below will process a single image: 1936 Olney Springs via figshare.  This code will eventually be turned into chunks of .py files and full of loops, but this will give you an idea of how the workflow currently stands.  For this workflow, you will need:

CELL 2:
- an image to georeference 
- a path to that image named `input_path`
- an output path for the processed image named `output_path`

CELL 4:
- the lat/long centroid coordinates
- the estimated pixel size, positive and negative form 


In [1]:
# Import packages
import os
from glob import glob
from osgeo import gdal, osr

import pyproj
from pyproj import Proj

import earthpy as et

In [2]:
# Download data
url = "https://ndownloader.figshare.com/files/22694774"
et.data.get_data(url=url, replace=True)

# Set working directory
os.chdir(os.path.join(et.io.HOME, 'earth-analytics'))

# Set paths to image pre-processed and post-processed
input_path = os.path.join('data','earthpy-downloads','ag274055.tif')
output_path = os.path.join('2020_fellowship', 'outputs', 'olneySprings_georef.tiff')

In [3]:
# Import source image
src_ds = gdal.Open(input_path)

# Describe source image size
x_height = src_ds.RasterXSize
y_width = src_ds.RasterYSize
print(x_height, y_width)

6089 5032


In [4]:
# Convert decimal degree centroids to UTMs (testing with single lon/lat)
# Centroid of yl022068.jpg via spreadsheet
lon = -103.94388889
lat = 38.16944444

myProj = Proj("+proj=utm +zone=13N, +north +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
easting,northing = myProj(lon, lat) 
print('Centroid UTM coordinates: ', easting, northing)

# Counting with pixels
pixel_size_estim = 0.950938
neg_pixel_size_estim = -0.950938
top_img_pixel = ((x_height/2)-1)
left_img_pixel = ((y_width/2)-1)

# Math to get to top left corner easting coordinate from centroid
x_topleft = easting - (pixel_size_estim/2) - (pixel_size_estim * top_img_pixel)
y_topleft = northing + (pixel_size_estim/2) + (pixel_size_estim * left_img_pixel)
print('Top left UTM coordinates: ', x_topleft, y_topleft)

Centroid UTM coordinates:  592511.2279902438 4225142.443000773
Top left UTM coordinates:  589616.5727182438 4227534.527539773


In [5]:
# Reformatting the image to geotiff
format = 'GTiff'
driver = gdal.GetDriverByName(format)

# Create copy with new name
dst_ds = driver.CreateCopy(output_path, src_ds, 0)

# Set top left corner coordinates in UTM with pixel size and rotation
gt = [x_topleft, pixel_size_estim, 0, y_topleft, 0, neg_pixel_size_estim]
dst_ds.SetGeoTransform(gt)

# Assign CRS
epsg = 32613 #utm zone 13n
srs = osr.SpatialReference()
srs.ImportFromEPSG(epsg)
dst_wkt = srs.ExportToWkt()
dst_ds.SetProjection(dst_wkt)

# Close and finalize dst_ds 
dst_ds = None 
src_ds = None